In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -qq ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.1 MB/s eta 0:00:00


In [3]:
!pip install -qq  hyperopt==0.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.0/966.0 kB 11.9 MB/s eta 0:00:00


In [4]:
#!git clone https://github.com/zhouhaoyi/Informer2020.git
!git clone -b test https://github.com/ai4infra/Time-Series-Library.git


Cloning into 'Time-Series-Library'...
remote: Enumerating objects: 1066, done.
remote: Counting objects: 100% (603/603), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1066 (delta 551), reused 535 (delta 535), pack-reused 463
Receiving objects: 100% (1066/1066), 21.42 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (773/773), done.


In [1]:
import pandas as pd
full_data = pd.read_csv("/content/drive/MyDrive/Marjan_ceph_metrics_5min.csv")
#full_data=full_data[["ds","y"]]#
#full_data[["y"]] = full_data[["y"]]
#full_data = full_data[['timestamp', 'capacity_instance_ceph_osd_latency_sec_apply_avg']].head(11000)
#full_data.columns = ["date","AL", "CL", "OR", "OW", "BR", "BW"]
# full_data.ds = pd.to_datetime(full_data.date)
# full_data = full_data.drop_duplicates(subset=['date'])
#full_data = full_data.fillna(method='ffill')
#full_data.columns = ["date", "LT"]
full_data = full_data[["date","AL", "CL", "OR", "OW", "BR", "BW"]]
full_data.info()
#full_data = full_data[["date", "LT"]] #.head(9999)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16129 entries, 0 to 16128
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    16129 non-null  object 
 1   AL      16129 non-null  float64
 2   CL      16129 non-null  float64
 3   OR      16129 non-null  float64
 4   OW      16129 non-null  float64
 5   BR      16129 non-null  float64
 6   BW      16129 non-null  float64
dtypes: float64(6), object(1)
memory usage: 882.2+ KB


In [6]:
full_data.to_csv("ceph-latency.csv")

## CEPH Latency with Informer

In [11]:
import sys
if not 'Time-Series-Library' in sys.path:
    sys.path += ['Time-Series-Library']
    %cd Time-Series-Library

In [ ]:
!ls

data	    environment.yml  img      main_informer.py	models	   requirements.txt  utils
Dockerfile  exp		     LICENSE  Makefile		README.md  scripts


In [ ]:
!pip install -r requirements.txt
#!pip install fastapi kaleido python-multipart uvicorn scipy==1.10.1

In [13]:
import argparse
import os
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
#from exp.exp_imputation import Exp_Imputation
#from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
#from exp.exp_anomaly_detection import Exp_Anomaly_Detection
#from exp.exp_classification import Exp_Classification
import random
import numpy as np
import argparse
import os
import torch
import tempfile
import random
import numpy as np
import os
import argparse
from filelock import FileLock
import tempfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import ray
from ray import train, tune
from ray.train import Checkpoint
from ray.tune.schedulers import AsyncHyperBandScheduler

In [4]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

In [5]:
model_name = "Informer"

In [6]:
args.task_name="long_term_forecast"
args.data_path="ceph-latency.csv"
args.root_path="/content/"
args.is_training=1
args.model_id="CEPHm1_96_96"
args.model=model_name
args.data="custom"
args.target="CL"
args.features="M"
args.seq_len=96
args.label_len=24
args.pred_len=96
args.d_model=512
args.n_heads=8
args.e_layers=3
args.d_layers=2
args.enc_in=1
args.dec_in=1
args.c_out=1
args.d_ff=2048
args.freq="t"
args.moving_avg=25
args.factor=8
args.embed='timeF'
args.distil=True
args.des='Exp'
args.checkpoints=f"saved_model/{model_name}"
args.seasonal_patterns="Daily"
args.inverse= False #True
args.anomaly_ratio=0.25
args.mask_ratio=0.25
args.top_k=5
args.num_kernels=6
args.dropout=0.02
args.activation="gelu"
args.output_attention=False
args.num_workers=10
args.itr=1
args.train_epochs=300
args.batch_size=32
args.patience=3
args.learning_rate=1e-4
args.loss ="MSE"
args.lradj="type2"
args.use_amp=False
# GPU
args.use_gpu = True if torch.cuda.is_available() else False
args.gpu=0
args.use_multi_gpu=False
args.devices='0,1,2,3'
args.p_hidden_dims=[128, 128]
args.p_hidden_layers=2


In [7]:
if args.task_name == 'long_term_forecast':
        Exp = Exp_Long_Term_Forecast

In [25]:
def train_func1(config):
    args = dotdict()
    ii=0
    args.task_name="long_term_forecast"
    args.data_path="ceph-latency.csv"
    args.root_path="/content/"
    args.is_training=1
    args.model_id="CEPHm1_96_96"
    args.model=model_name
    args.data="custom"
    args.target="CL"
    args.features="M"
    args.seq_len=96
    args.label_len=24
    args.pred_len=96
    args.d_model=512
    args.n_heads=8
    args.e_layers=3
    args.d_layers=2
    args.enc_in=1
    args.dec_in=1
    args.c_out=1
    args.d_ff=2048
    args.freq="t"
    args.moving_avg=25
    args.factor=8
    args.embed='timeF'
    args.distil=True
    args.des='Exp'
    args.checkpoints=f"saved_model/{model_name}"
    args.seasonal_patterns="Daily"
    args.inverse= False #True
    args.anomaly_ratio=0.25
    args.mask_ratio=0.25
    args.top_k=5
    args.num_kernels=6
    args.dropout=0.02
    args.activation="gelu"
    args.output_attention=False
    args.num_workers=10
    args.itr=1
    args.train_epochs=300
    args.batch_size=32
    args.patience=3
    args.learning_rate=1e-4
    args.loss ="MSE"
    args.lradj="type2"
    args.use_amp=False
    # GPU
    args.use_gpu = True if torch.cuda.is_available() else False
    args.gpu=0
    args.use_multi_gpu=True
    args.devices='0,1,2,3'
    args.p_hidden_dims=[128, 128]
    args.p_hidden_layers=2

    # GPU
    args.use_gpu = True
    args.gpu=0
    args.use_multi_gpu=False
    args.devices='0,1,2,3'
    args.detail_freq="t"
    should_checkpoint = config.get("should_checkpoint", False)
    args.n_heads=config["n_heads"]
    args.e_layers=config["e_layers"]
    args.d_layers=config["d_layers"]
    args.factor= config["factor"]
    args.seq_len=config["seq_len"]
    args.label_len=config["label_len"]
    args.pred_len=config["pred_len"]
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data,
                                                                                                         args.features,
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed,
                args.distil, args.mix, args.des, ii)

    exp =  Exp_Long_Term_Forecast(args)
    while True:
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
         # set experiments
        exp.train(setting)
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        metrics = exp.test(setting)
        #metrics = metrics
        # Report metrics (and possibly a checkpoint)
        if should_checkpoint:
            with tempfile.TemporaryDirectory() as tempdir:
                torch.save(model.state_dict(), os.path.join(tempdir, "model.pt"))
                train.report(metrics, checkpoint=Checkpoint.from_directory(tempdir))
        else:
            train.report(metrics)




In [27]:
ray.shutdown()

In [28]:
args.cuda= True
args.smoke_test = False
ray.init(num_cpus=2 if args.smoke_test else None)

2023-11-20 08:11:08,420	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.8.0


In [ ]:
!export RAY_AIR_NEW_OUTPUT=0
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search import ConcurrencyLimiter


# for early stopping
sched = AsyncHyperBandScheduler()

initial_params = [{
            "n_heads":  8,
            "e_layers": 2,
            "d_layers": 2,
            "seq_len": 24,
            "label_len": 24,
            "pred_len": 24,
            "factor": 3,

}]
resources_per_trial = {"cpu": 2, "gpu": 0.4}
algo = HyperOptSearch(points_to_evaluate=initial_params)
algo = ConcurrencyLimiter(algo, max_concurrent=4)

tuner = tune.Tuner(
        tune.with_resources(train_func1, resources=resources_per_trial),
        tune_config=tune.TuneConfig(
            metric="mse",
            mode="min",
            scheduler=sched,
            num_samples= 50,
            search_alg=algo,
        ),
        run_config=train.RunConfig(
            name=model_name,
            stop={
                "mse": 0.0005,
                "training_iteration": 100,
            },
        ),
        #  param_space={
        #     "n_heads":  tune.choice([8,16]),
        #     "e_layers":  tune.choice([6, 4, 3, 2]),
        #     "d_layers": tune.choice([2]),
        #     "seq_len": tune.choice([24, 48, 96, 192, 288, 480, 672]),
        #     "label_len": tune.choice([24, 48, 96, 192, 288, 480, 672]),
        #     "pred_len": tune.choice([96, 192, 288, 480, 672]),
        #     "factor": tune.choice([3,5,8,10]),

        # },
        param_space={
            "n_heads":  tune.choice([8]),
            "e_layers":  tune.choice([2]),
            "d_layers": tune.choice([2]),
            "seq_len": tune.choice([24]),
            "label_len": tune.choice([24]),
            "pred_len": tune.choice([24]),
            "factor": tune.choice([3]),
        },
      )
results = tuner.fit()

print("Best config is:", results.get_best_result().config)

2023-11-20 08:11:11,079	INFO tune.py:595 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+----------------------------------------------------------+
| Configuration for experiment     Informer                |
+----------------------------------------------------------+
| Search algorithm                 SearchGenerator         |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 50                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/Informer
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/Informer`

Trial status: 1 PENDING
Current time: 2023-11-20 08:11:11. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs
+-------------------------------------------------------------------------------------------------------------------------+
| Trial name             status       n_heads     e_layers     d_layers     seq_len     label_len     pred_len     factor |
+-------------------------------------------

In [ ]:
#!bash ./scripts/long_term_forecast/ceph/Autoformer.sh

## Experiments: Train and Test

In [ ]:
# from data.data_loader import Dataset_Custom
# from torch.utils.data import DataLoader
# import pandas as pd
# import os

In [ ]:
# args = dotdict()
# # we take ETTh2 as an example
# args.root_path = '../content'
# args.data_path = 'ceph-latency.csv'

# df = pd.read_csv(os.path.join(args.root_path, args.data_path))


In [ ]:
# df["LT"].plot()

In [ ]:
# df = df[["date", "LT"]]
# df.describe()

In [ ]:
# df.info()

In [ ]:
# '''
# We set 'HULL' as target instead of 'OT'

# The following frequencies are supported:
#         Y   - yearly
#             alias: A
#         M   - monthly
#         W   - weekly
#         D   - daily
#         B   - business days
#         H   - hourly
#         T   - minutely
#             alias: min
#         S   - secondly
# '''

# args.target = 'LT'
# args.freq = 't'
# args.seq_len = 96
# args.label_len = 48
# args.pred_len = 24
# args.features = "S"
# args.num_workers = 0

# Data = Dataset_Custom
# timeenc = 0 if args.embed!='timeF' else 1
# flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1

# data_set = Data(
#     root_path=args.root_path,
#     data_path=args.data_path,
#     flag=flag,
#     size=[args.seq_len, args.label_len, args.pred_len],
#     features=args.features,
#     timeenc=timeenc,
#     target=args.target, # HULL here
#     freq="t" # 'h': hourly, 't':minutely
# )
# data_loader = DataLoader(
#     data_set,
#     batch_size=batch_size,
#     shuffle=shuffle_flag,
#     num_workers=args.num_workers,
#     drop_last=drop_last)

In [ ]:
# #args = dotdict()

# args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

# args.data = 'custom' # data
# #args.root_path = './ETDataset/ETT-small/' # root path of data file
# #args.data_path = 'ETTh1.csv' # data file
# #args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
# #args.target = 'OT' # target feature in S or MS task
# #args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
# args.checkpoints = './CEPH_informer_checkpoints' # location of model checkpoints

# args.seq_len = 96 # input sequence length of Informer encoder
# args.label_len = 48 # start token length of Informer decoder
# args.pred_len = 24 # prediction sequence length
# # Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

# args.enc_in = 7 # encoder input size
# args.dec_in = 7 # decoder input size
# args.c_out = 7 # output size
# args.factor = 5 # probsparse attn factor
# args.d_model = 512 # dimension of model
# args.n_heads = 8 # num of heads
# args.e_layers = 3 #2 # num of encoder layers
# args.d_layers = 2 # # num of decoder layers
# args.d_ff = 2048 # dimension of fcn in model
# args.dropout = 0.05 # dropout
# args.attn = 'prob' # attention used in encoder, options:[prob, full]
# args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
# args.activation = 'gelu' # activation
# args.distil = True # whether to use distilling in encoder
# args.output_attention = False # whether to output attention in ecoder
# args.mix = True
# args.padding = 0
# args.freq = 't'

# args.batch_size = 32
# args.learning_rate = 0.0001
# args.loss = 'mse'
# args.lradj = 'type1'
# args.use_amp = False # whether to use automatic mixed precision training

# args.num_workers = 0
# args.itr = 1
# args.train_epochs = 25 #50
# args.patience = 5
# args.des = 'exp'

# args.use_gpu = True if torch.cuda.is_available() else False
# args.gpu = 0

# args.use_multi_gpu = False
# args.devices = '0,1,2,3'


In [ ]:
# args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

# if args.use_gpu and args.use_multi_gpu:
#     args.devices = args.devices.replace(' ','')
#     device_ids = args.devices.split(',')
#     args.device_ids = [int(id_) for id_ in device_ids]
#     args.gpu = args.device_ids[0]

In [ ]:
# # Set augments by using data name
# data_parser = {
#     'custom':{'data':'ceph-latency.csv','T':'LT','S':[1,1,1]},

# }
# if args.data in data_parser.keys():
#     data_info = data_parser[args.data]
#     args.data_path = data_info['data']
#     args.target = data_info['T']
#     args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [ ]:
# args.detail_freq = args.freq
# args.freq = args.freq[-1:]

In [ ]:
# print('Args in experiment:')
# print(args)

In [ ]:
# Exp = Exp_Informer

In [ ]:
# for ii in range(args.itr):
#     # setting record of experiments
#     setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features,
#                 args.seq_len, args.label_len, args.pred_len,
#                 args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

#     # set experiments
#     exp = Exp(args)

#     # train
#     print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
#     exp.train(setting)

#     # test
#     print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
#     exp.test(setting)

#     torch.cuda.empty_cache()

In [ ]:
# import os

# # set saved model path
# setting = 'long_term_forecast_CEPHm1_96_96_Autoformer_custom_ftS_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0'
# path = os.path.join(args.checkpoints,setting,'checkpoint.pth')

In [ ]:
# # If you already have a trained model, you can set the arguments and model path, then initialize a Experiment and use it to predict
# # Prediction is a sequence which is adjacent to the last date of the data, and does not exist in the data
# # If you want to get more information about prediction, you can refer to code `exp/exp_informer.py function predict()` and `data/data_loader.py class Dataset_Pred`

# exp = Exp(args)

# exp.predict(setting, True)

In [ ]:
# # the prediction will be saved in ./results/{setting}/real_prediction.npy
# import numpy as np

# prediction = np.load('./results/'+setting+'/real_prediction.npy')

# prediction.shape

In [ ]:
# # here is the detailed code of function predict

# def predict(exp, setting, load=False):
#     pred_data, pred_loader = exp._get_data(flag='pred')

#     if load:
#         path = os.path.join(exp.args.checkpoints, setting)
#         best_model_path = path+'/'+'checkpoint.pth'
#         exp.model.load_state_dict(torch.load(best_model_path))

#     exp.model.eval()

#     preds = []

#     for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(pred_loader):
#         batch_x = batch_x.float().to(exp.device)
#         batch_y = batch_y.float()
#         batch_x_mark = batch_x_mark.float().to(exp.device)
#         batch_y_mark = batch_y_mark.float().to(exp.device)

#         # decoder input
#         if exp.args.padding==0:
#             dec_inp = torch.zeros([batch_y.shape[0], exp.args.pred_len, batch_y.shape[-1]]).float()
#         elif exp.args.padding==1:
#             dec_inp = torch.ones([batch_y.shape[0], exp.args.pred_len, batch_y.shape[-1]]).float()
#         else:
#             dec_inp = torch.zeros([batch_y.shape[0], exp.args.pred_len, batch_y.shape[-1]]).float()
#         dec_inp = torch.cat([batch_y[:,:exp.args.label_len,:], dec_inp], dim=1).float().to(exp.device)
#         # encoder - decoder
#         if exp.args.use_amp:
#             with torch.cuda.amp.autocast():
#                 if exp.args.output_attention:
#                     outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
#                 else:
#                     outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
#         else:
#             if exp.args.output_attention:
#                 outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
#             else:
#                 outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
#         f_dim = -1 if exp.args.features=='MS' else 0
#         batch_y = batch_y[:,-exp.args.pred_len:,f_dim:].to(exp.device)

#         pred = outputs.detach().cpu().numpy()#.squeeze()

#         preds.append(pred)

#     preds = np.array(preds)
#     preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])

#     # result save
#     folder_path = './results/' + setting +'/'
#     if not os.path.exists(folder_path):
#         os.makedirs(folder_path)

#     np.save(folder_path+'real_prediction.npy', preds)

#     return preds


In [ ]:
# # you can also use this prediction function to get result
# prediction = predict(exp, setting, True)

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.plot(prediction[0,:,-1])
# plt.show()

In [ ]:
# from data.data_loader import Dataset_Pred
# from torch.utils.data import DataLoader

In [ ]:
# Data = Dataset_Pred
# timeenc = 0 if args.embed!='timeF' else 1
# flag = 'pred'; shuffle_flag = False; drop_last = False; batch_size = 1

# freq = args.detail_freq

# data_set = Data(
#     root_path=args.root_path,
#     data_path=args.data_path,
#     flag=flag,
#     size=[args.seq_len, args.label_len, args.pred_len],
#     features=args.features,
#     target=args.target,
#     timeenc=timeenc,
#     freq=freq
# )
# data_loader = DataLoader(
#     data_set,
#     batch_size=batch_size,
#     shuffle=shuffle_flag,
#     num_workers=args.num_workers,
#     drop_last=drop_last)

In [ ]:
# len(data_set), len(data_loader)

## Visualization

In [ ]:
# # When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# # The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

# #preds = np.load('./results/'+setting+'/pred.npy')
# preds = np.load('./results/'+setting+'/real_prediction.npy')
# trues = np.load('./results/'+setting+'/true.npy')

# # [samples, pred_len, dimensions]
# preds.shape, trues.shape

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
# # draw LT prediction
# plt.figure()
# plt.plot(trues[0,:,-1], label='GroundTruth')
# plt.plot(preds[0,:,-1], label='realPrediction')
# plt.plot(prediction[0,:,-1], label='Prediction')
# plt.legend()
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(trues[0,:,0], label='GroundTruth')
# plt.plot(preds[0,:,0], label='Prediction')
# plt.legend()
# plt.show()